In [ ]:

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, losses
import matplotlib.pyplot as plt


In [ ]:

# ссылка на скачивание датасета
link = "https://archive.ics.uci.edu/static/public/17/breast+cancer+wisconsin+diagnostic.zip"
# сформируем имена колонок
# имена признаков, которые повторяются три раза
features = [
    "radius",
    "texture",
    "perimeter",
    "area",
    "smoothness",
    "compactness",
    "concavity",
    "concave_points",
    "symmetry",
    "fractal_dimension"
]
# повторяем имена признаков три раза с разными индексами 
# и добавляем имена первых двух колонок с помощью 
# контатенации списков оператором `+`
# ["id", "class"] + ["radius-1", "area-1", ...] + ["radius-2", "area-2", ...
names = sum(([f"{f}-{i}" for f in features] for i in range(1, 4)), ['id', 'class'])
raw_data = pd.read_csv(link, names=names)
# посмотрим на результат
raw_data


In [ ]:

# Слегка преобработаем данные
# 1. Уберем колонку id
data = raw_data.drop(["id"], axis=1)
# 2. Закодируем метку числом 0 и 1 
data["class"], class_labels = pd.factorize(data["class"])
print("Class labels:", class_labels)
# посмотрим на результат
data

In [ ]:
from sklearn.model_selection import train_test_split
train_x, test_x = train_test_split(data, test_size=0.2, random_state=2023)
train_y, test_y = train_x.pop("class"), test_x.pop("class")
# посмотрим на результат
train_x

In [ ]:

# создаем модель keras
base_model = tf.keras.Sequential(
    layers=[ # описываем слои нейронной сети
        layers.Dense( # создаем полносвязный слой
            16, # количество нейронов 
            activation='relu', # функция активации
            input_dim=train_x.shape[1]
        ), # количество нейронов на входе, 
        # нужно тольно для первого слоя
        layers.Dense(8, activation='relu'), # так же задаем второй слой
        layers.Dense(6, activation='relu'),
        layers.Dense(1, activation='sigmoid') # выходной слой
    ],
    name="breast-cancer-base-model",
)
# посмотрим описание модели
base_model.summary()

In [ ]:

base_model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [ ]:
base_model_history = base_model.fit(
    train_x, train_y,
    epochs=40,
    batch_size=20,
    validation_data=(test_x, test_y)
)

In [ ]:

from typing import Dict, List, Tuple
def plot_history(
                history: Dict[str, List[float]],
                title: str = "",
                metric_name: str = 'loss',
                ylim: Tuple[float, float] = None
    ):

    train_values = history[metric_name]
    plt.plot(train_values, label=f'Train {metric_name}')
    try:
        val_values = history['val_' + metric_name]
        plt.plot(val_values, label=f'Validation {metric_name}')
    except KeyError:
        val_values = []
    plt.title(title)
    all_values = train_values + val_values
    ylim = ylim or (0.9 * min(all_values), 1.1 * max(all_values))
    plt.ylim(ylim)
    plt.ylabel(metric_name.capitalize())
    plt.xlabel("Epoch")
    plt.legend(loc='best')
    plt.show()


In [ ]:

plot_history(base_model_history.history, "Основная модель", "accuracy")


In [ ]:

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, losses, datasets
import matplotlib.pyplot as plt

In [ ]:
(train_x, train_y), (test_x, test_y) = datasets.mnist.load_data()


In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
 plt.subplot(5,5,i+1)
 plt.xticks([])
 plt.yticks([])
 plt.grid(False)
 plt.imshow(train_x[i])
 plt.xlabel(str(train_y[i]))
plt.show()

In [ ]:

base_model = tf.keras.Sequential(
    [
        layers.Flatten(input_shape=(28, 28, 1)),
        layers.Rescaling(1 / 255),
        layers.Dense(128, activation='relu'),
        layers.Dense(32, activation='relu'),
        layers.Dense(10),
        layers.Softmax()
    ],
    name="mnist-base-model",
)
# посмотрим описание модели
base_model.summary()

In [ ]:

base_model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:

%%time
base_model_history = base_model.fit(
    train_x, train_y,
    epochs=40,
    batch_size=100,
    validation_data=(test_x, test_y)
)

In [ ]:

plot_history(base_model_history.history, "Основная модель")

In [ ]:

base_model_history.history['val_accuracy'][-1]

In [ ]:

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/aut
url = https://archive.ics.uci.edu/static/public/9/auto+mpg.zip
column_names = ['MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight',
 'Acceleration', 'Model Year', 'Origin']
raw_dataset = pd.read_csv(url, names=column_names,
 na_values='?', comment='\t',
 sep=' ', skipinitialspace=True)
